In [7]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [2]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)

In [3]:
df.drop(df.index[:1], inplace=True)

In [8]:
df.loc[df['name'] == 'Michael Roth']

,Unnamed: 0,datum,redeID,vorname,nachname,fraktion,name,inhalt_neu,"Jens Ackermann;""FDP"";1975;""Sachsen-Anhalt"";;""2. Juli 1975"";""http://www.jens-ackermann.info"";""jens.ackermann@bundestag.de"""
328,48.0,28.02.2018,ID191604800,Michael,Roth,SPD,Michael Roth,"vielen dank, herr präsident. liebe kolleginnen...",NaN
331,51.0,28.02.2018,ID191605100,Michael,Roth,SPD,Michael Roth,verehrte frau präsidentin! liebe kolleginnen u...,NaN
702,67.0,19.04.2018,ID192606700,Michael,Roth,SPD,Michael Roth,"guten tag, frau präsidentin! liebe kolleginnen...",NaN
1343,97.0,16.03.2022,ID202009700,Michael,Roth,SPD,Michael Roth,liebe frau präsidentin! kolleginnen und kolleg...,NaN
3806,63.0,07.06.2018,ID193606300,Michael,Roth,SPD,Michael Roth,"guten tag, liebe frau präsidentin! liebe kolle...",NaN
5093,16.0,15.03.2019,ID198701600,Michael,Roth,SPD,Michael Roth,"guten morgen, herr präsident! liebe kolleginne...",NaN
5478,88.0,11.04.2019,ID199508800,Michael,Roth,SPD,Michael Roth,vielen dank. lieber herr präsident! liebe koll...,NaN
9181,15.0,02.10.2020,ID1918101500,Michael,Roth,SPD,Michael Roth,herr präsident! liebe kolleginnen und kollegen...,NaN
9599,77.0,21.05.2021,ID1923107700,Michael,Roth,SPD,Michael Roth,"guten tag, liebe frau präsidentin! liebe kolle...",NaN
10100,72.0,22.04.2020,ID1915507200,Michael,Roth,SPD,Michael Roth,sehr geehrter herr präsident! sehr geehrte dam...,NaN
